In [1]:
import pandas as pd
import pyodbc
import os
import json

In [2]:
%store -r server
%store -r database
%store -r username
%store -r password
%store -r driver
%store -r database_table_name

In [3]:
import pyodbc

# Define your connection parameters
# server = 'your_server_name.database.windows.net'
# database = 'your_database_name'
# username = 'your_username'
# password = 'your_password'

# Create the connection string
connection_string= f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Establish a connection
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    print("Connected to the database.")
except Exception as e:
    print(f"Connection failed: {str(e)}")
    conn = None


Connected to the database.


In [94]:
import pyodbc
import pandas as pd
import time

# Define your Azure SQL database connection parameters
server = server
database = database
username = username
password = password
driver = driver
table_name = database_table_name

# Define your SQL query
sql_query = f"SELECT * FROM {table_name}"
sql_query = f"""SELECT *
FROM {table_name}
WHERE day_posted >= DATEADD(month, -1, GETDATE())"""


# Set the maximum number of retry attempts and the delay between retries
max_retries = 3
retry_delay_seconds = 5

for attempt in range(max_retries):
    try:
        # Establish a connection to the database
        conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')
        cursor = conn.cursor()
        print("Connected to database")

        # Execute the SQL query
        cursor.execute(sql_query)

        # Fetch the query results into a list of tuples
        rows = cursor.fetchall()

        # Create a DataFrame from the query results
        df = pd.DataFrame([tuple(row) for row in rows], columns=[desc[0] for desc in cursor.description])

        print("Data retrieved and stored in DataFrame:")
        print(df.head())

        # Break out of the loop if the query was successful
        break

    except pyodbc.Error as e:
        print(f"Error executing SQL query (Attempt {attempt + 1}): {str(e)}")

        # If this is not the last attempt, wait for a delay before retrying
        if attempt < max_retries - 1:
            print(f"Retrying in {retry_delay_seconds} seconds...")
            time.sleep(retry_delay_seconds)

    finally:
        if 'cursor' in locals():
            cursor.close()
        

Connected to database
Data retrieved and stored in DataFrame:
                   Job_Name                       Generic_Job_Name  \
0  Senior Account Associate                      financial officer   
1                  Marketer                              Not given   
2         Junior Accountant                      financial officer   
3                   Cleaner                      vocational worker   
4                HR Officer  human resource officer/hiring manager   

                               Company_Name                        Industry  \
0       Jobberman (Third Party Recruitment)  Accounting, Auditing & Finance   
1  Ancheng Secure Solutions Nigeria Limited      Marketing & Communications   
2                            Medife Limited  Accounting, Auditing & Finance   
3                     GRAND SWISS HOTEL LTD                 Health & Safety   
4                           ABBSTAL LIMITED                 Human Resources   

  Location   Job_Type                     

In [95]:
df["Day_Posted"] = pd.to_datetime(df["Day_Posted"])

df.sort_values("Day_Posted", ascending=False)

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url
1066,Busser,vocational worker,GRAND SWISS HOTEL LTD,Food Services & Catering,Anambra,Full Time,"A Busser, or Food Runner, is in charge of sett...",High School (S.S.C.E),Entry level,No Experience/Less than 1 year,"Removing used plates, glasses, cutlery and nap...",75000,"Less than 75,000",Today,2023-10-18,2023-10-18 14:13:53,https://www.jobberman.com/listings/busser-np5krv
1064,Accountant,financial officer,Jobberman (Third Party Recruitment),"Accounting, Auditing & Finance",Lagos,Full Time,Our client is looking to hire a suitable candi...,HND,Mid level,3 years,Perform general accounting activities includin...,Not given,Not given,Today,2023-10-18,2023-10-18 14:13:27,https://www.jobberman.com/listings/accountant-...
1071,Science Subject Tutor,teacher,Myschool Limited,"Research, Teaching & Training",Lagos,Full Time,Myschool is looking for passionate and motivat...,N.C.E,Entry level,1 year,Developing and managing academic content mainl...,75000,"75,000 - 150,000",Today,2023-10-18,2023-10-18 14:14:22,https://www.jobberman.com/listings/tutor-for-s...
530,Restaurant Manager,hospitality/cullinary personnel,Royal waterfront suites,Admin & Office,Lagos,Full Time,We are looking for a Restaurant Manager to lea...,HND,Mid level,2 years,Coordinate daily Front of the House and Back o...,75000,"75,000 - 150,000",Today,2023-10-18,2023-10-18 14:14:10,https://www.jobberman.com/listings/restaurant-...
1069,Restaurant Manager,hospitality/cullinary personnel,Royal waterfront suites,Admin & Office,Lagos,Full Time,We are looking for a Restaurant Manager to lea...,HND,Mid level,2 years,Coordinate daily Front of the House and Back o...,75000,"75,000 - 150,000",Today,2023-10-18,2023-10-18 14:14:10,https://www.jobberman.com/listings/restaurant-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3050,Mechanical Engineer (Power Plant),architect/engineer,Dangote Industries Limited,Engineering & Technology,Rest of Nigeria,Full Time,,Not given,Not given,Not given,"\nJob Summary\nPlan, schedule, and supervise t...",Not given,Not given,2 weeks ago,2023-09-24,2023-10-08 07:55:36,https://www.jobberman.com/listings/mechanical-...
3051,IngÃ©nieur.e Achats Energies (H/F),other,ENGIE Laborelec,Medical & Pharmaceutical,Rest of Nigeria,Full Time,,Not given,Not given,Not given,\n IngÃ©nieur.e Achats Energies (H/F) Identif...,Not given,Not given,2 weeks ago,2023-09-24,2023-10-08 07:55:40,https://www.jobberman.com/listings/inganieure-...
3052,"Payroll Expert, Sub-Saharan Africa",Human Resource Officer/Hiring Manager,Deel,Admin & Office,Rest of Nigeria,Full Time,,Not given,Not given,Not given,\n Who we are and what we do Deel is a glob...,Not given,Not given,2 weeks ago,2023-09-24,2023-10-08 07:55:45,https://www.jobberman.com/listings/payroll-exp...
3053,Money Market Trader,stocks/fx trader,Standard Bank Group,"Accounting, Auditing & Finance",Lagos,Full Time,,Not given,Not given,Not given,\n Job Description To explore all domestic ...,Not given,Not given,2 weeks ago,2023-09-24,2023-10-08 07:55:49,https://www.jobberman.com/listings/money-marke...


In [96]:
df

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url
0,Senior Account Associate,financial officer,Jobberman (Third Party Recruitment),"Accounting, Auditing & Finance",Lagos,Full Time,"As a senior account associate, you support the...",Degree,Senior level,5 years,Support and assist the COO in firm and client...,Not given,Not given,3 weeks ago,2023-09-27,2023-10-18 14:01:44,https://www.jobberman.com/listings/senior-acco...
1,Marketer,Not given,Ancheng Secure Solutions Nigeria Limited,Marketing & Communications,Abuja,Full Time,We are looking for Marketers that will be resp...,HND,Entry level,2 years,Conduct market research to identify customer n...,150000,"150,000 - 250,000",6 days ago,2023-10-12,2023-10-18 14:02:03,https://www.jobberman.com/listings/marketer-6d...
2,Junior Accountant,financial officer,Medife Limited,"Accounting, Auditing & Finance",Lagos,Full Time,We are searching for a highly organized and mo...,Degree,Entry level,1 year,Maintain accurate and organized records of all...,Not given,Not given,6 days ago,2023-10-12,2023-10-18 14:02:08,https://www.jobberman.com/listings/junior-acco...
3,Cleaner,vocational worker,GRAND SWISS HOTEL LTD,Health & Safety,Anambra,Full Time,We are looking to hire a suitable candidate to...,High School (S.S.C.E),Entry level,No Experience/Less than 1 year,Coordinating with other Cleaners to keep your ...,75000,"Less than 75,000",Today,2023-10-18,2023-10-18 14:00:45,https://www.jobberman.com/listings/cleaner-wpm9q6
4,HR Officer,human resource officer/hiring manager,ABBSTAL LIMITED,Human Resources,Lagos,Full Time,We are looking for a skilled HR Officer who wi...,Degree,Mid level,2 years,Support the development and implementation of ...,75000,"75,000 - 150,000",6 days ago,2023-10-12,2023-10-18 14:02:14,https://www.jobberman.com/listings/hr-officer-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3504,Digital Marketing Specialist,Not given,Cybertech Digitals,Marketing & Communications,Work From Home,Full Time,As a Digital Marketing Specialist for one of o...,Diploma,Mid level,3 years,Develop and execute end-to-end digital marketi...,150000,"150,000 - 250,000",6 days ago,2023-10-12,2023-10-18 14:01:31,https://www.jobberman.com/listings/digital-mar...
3505,Content Creator,Not given,Bantupage Ltd.,Marketing & Communications,Lagos,Contract,A highly experienced Content Creator who knows...,OND,Mid level,3 years,Create content and skits from videos and photo...,75000,"75,000 - 150,000",6 days ago,2023-10-12,2023-10-18 14:01:57,https://www.jobberman.com/listings/content-cre...
3506,Finance Lead,financial officer,Jobberman (Third Party Recruitment),"Accounting, Auditing & Finance",Lagos,Full Time,We seek a Finance Lead to run day-to-day under...,Degree,Mid level,3 years,"Manage the company's finances, budgeting, fore...",Not given,Not given,1 week ago,2023-10-11,2023-10-18 14:01:50,https://www.jobberman.com/listings/finance-lea...
3507,Credit Officer,financial officer,PERFECT RESULT PREVERSITY CONSULT LIMITED,Management & Business Development,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,OND,Entry level,2 years,"Review loan requests, collect daily contributi...",75000,"Less than 75,000",Today,2023-10-18,2023-10-18 14:14:33,https://www.jobberman.com/listings/credit-offi...


In [122]:
df["Salary_Range"].value_counts()

Not given                1641
75,000 - 150,000          718
Less than 75,000          618
150,000 - 250,000         281
250,000 - 400,000         139
400,000 - 600,000          32
600,000 - 900,000          30
Commission Only            22
More than 12,000+          10
500 - 750                   4
2500 - 4000                 4
900,000 - 1,200,000         4
1,500,000 - 2,000,000       2
6000 - 8000                 2
3,000,000 - 4,000,000       2
Name: Salary_Range, dtype: int64

In [97]:
with open("./job_skills.json", "r") as json_file:
    
    skills_json = json_file.read()
    
    skills = json.loads(skills_json)
    
skills_ready = {key.lower(): value for key,value in skills.items()}

In [98]:
df["Job_Skills"] = df["Generic_Job_Name"].map(skills_ready)

df.head()

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url,Job_Skills
0,Senior Account Associate,financial officer,Jobberman (Third Party Recruitment),"Accounting, Auditing & Finance",Lagos,Full Time,"As a senior account associate, you support the...",Degree,Senior level,5 years,Support and assist the COO in firm and client...,Not given,Not given,3 weeks ago,2023-09-27,2023-10-18 14:01:44,https://www.jobberman.com/listings/senior-acco...,"[Financial analysis, Accounting, Budgeting, Au..."
1,Marketer,Not given,Ancheng Secure Solutions Nigeria Limited,Marketing & Communications,Abuja,Full Time,We are looking for Marketers that will be resp...,HND,Entry level,2 years,Conduct market research to identify customer n...,150000,"150,000 - 250,000",6 days ago,2023-10-12,2023-10-18 14:02:03,https://www.jobberman.com/listings/marketer-6d...,NaN
2,Junior Accountant,financial officer,Medife Limited,"Accounting, Auditing & Finance",Lagos,Full Time,We are searching for a highly organized and mo...,Degree,Entry level,1 year,Maintain accurate and organized records of all...,Not given,Not given,6 days ago,2023-10-12,2023-10-18 14:02:08,https://www.jobberman.com/listings/junior-acco...,"[Financial analysis, Accounting, Budgeting, Au..."
3,Cleaner,vocational worker,GRAND SWISS HOTEL LTD,Health & Safety,Anambra,Full Time,We are looking to hire a suitable candidate to...,High School (S.S.C.E),Entry level,No Experience/Less than 1 year,Coordinating with other Cleaners to keep your ...,75000,"Less than 75,000",Today,2023-10-18,2023-10-18 14:00:45,https://www.jobberman.com/listings/cleaner-wpm9q6,"[Specific skills, Vocational training, Trade s..."
4,HR Officer,human resource officer/hiring manager,ABBSTAL LIMITED,Human Resources,Lagos,Full Time,We are looking for a skilled HR Officer who wi...,Degree,Mid level,2 years,Support the development and implementation of ...,75000,"75,000 - 150,000",6 days ago,2023-10-12,2023-10-18 14:02:14,https://www.jobberman.com/listings/hr-officer-...,"[Recruitment strategies, Compensation and bene..."


In [99]:
df_clean = df.dropna()

df_clean

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url,Job_Skills
0,Senior Account Associate,financial officer,Jobberman (Third Party Recruitment),"Accounting, Auditing & Finance",Lagos,Full Time,"As a senior account associate, you support the...",Degree,Senior level,5 years,Support and assist the COO in firm and client...,Not given,Not given,3 weeks ago,2023-09-27,2023-10-18 14:01:44,https://www.jobberman.com/listings/senior-acco...,"[Financial analysis, Accounting, Budgeting, Au..."
2,Junior Accountant,financial officer,Medife Limited,"Accounting, Auditing & Finance",Lagos,Full Time,We are searching for a highly organized and mo...,Degree,Entry level,1 year,Maintain accurate and organized records of all...,Not given,Not given,6 days ago,2023-10-12,2023-10-18 14:02:08,https://www.jobberman.com/listings/junior-acco...,"[Financial analysis, Accounting, Budgeting, Au..."
3,Cleaner,vocational worker,GRAND SWISS HOTEL LTD,Health & Safety,Anambra,Full Time,We are looking to hire a suitable candidate to...,High School (S.S.C.E),Entry level,No Experience/Less than 1 year,Coordinating with other Cleaners to keep your ...,75000,"Less than 75,000",Today,2023-10-18,2023-10-18 14:00:45,https://www.jobberman.com/listings/cleaner-wpm9q6,"[Specific skills, Vocational training, Trade s..."
4,HR Officer,human resource officer/hiring manager,ABBSTAL LIMITED,Human Resources,Lagos,Full Time,We are looking for a skilled HR Officer who wi...,Degree,Mid level,2 years,Support the development and implementation of ...,75000,"75,000 - 150,000",6 days ago,2023-10-12,2023-10-18 14:02:14,https://www.jobberman.com/listings/hr-officer-...,"[Recruitment strategies, Compensation and bene..."
5,Communications and Public Relation Executive,social media personnel/marketer/digital marketer,Connected Analytics Limited(ThankUCash),Marketing & Communications,Lagos,Full Time,The Communications and PR Executive will help ...,Degree,Mid level,3 years,"Oversees and produces, from concept to complet...",250000,"250,000 - 400,000",6 days ago,2023-10-12,2023-10-18 14:02:21,https://www.jobberman.com/listings/pr-and-comm...,"[Social media marketing, Content creation, SEO..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3502,Property Intelligence Specialist,consultant,Oloruntoba olotu & Co,Software & Data,Work From Home,Full Time,We are looking for virtual driving for dollars...,Degree,Entry level,2 years,"Use online resources, databases, and maps to i...",75000,"75,000 - 150,000",4 days ago,2023-10-14,2023-10-18 14:07:02,https://www.jobberman.com/listings/property-in...,"[Problem-solving, Analysis, Client consulting,..."
3503,Content Writer,writer,Capsgain Solutions,Creative & Design,Lagos,Full Time,We are seeking a skilled and creative Content ...,Degree,Mid level,2 years,"Content Creation: Produce high-quality, origin...",75000,"75,000 - 150,000",1 week ago,2023-10-11,2023-10-18 14:01:25,https://www.jobberman.com/listings/content-wri...,"[Writing proficiency, Creative writing, Editin..."
3506,Finance Lead,financial officer,Jobberman (Third Party Recruitment),"Accounting, Auditing & Finance",Lagos,Full Time,We seek a Finance Lead to run day-to-day under...,Degree,Mid level,3 years,"Manage the company's finances, budgeting, fore...",Not given,Not given,1 week ago,2023-10-11,2023-10-18 14:01:50,https://www.jobberman.com/listings/finance-lea...,"[Financial analysis, Accounting, Budgeting, Au..."
3507,Credit Officer,financial officer,PERFECT RESULT PREVERSITY CONSULT LIMITED,Management & Business Development,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,OND,Entry level,2 years,"Review loan requests, collect daily contributi...",75000,"Less than 75,000",Today,2023-10-18,2023-10-18 14:14:33,https://www.jobberman.com/listings/credit-offi...,"[Financia

In [101]:
df_sorted = df_clean.sort_values("Industry")

#df_sorted = df_sorted.loc[df_sorted["Generic_Job_Name"] != "Other", :]

In [102]:
df_sorted.groupby(["Generic_Job_Name"])
df_sorted["Job_Skills"].dtype

dtype('O')

In [103]:
df_sorted.set_index("Industry")[ "Job_Skills"].to_dict()

{'Accounting, Auditing & Finance': ['Financial analysis',
  'Accounting',
  'Budgeting',
  'Auditing',
  'Financial reporting',
  'Risk management',
  'Investment',
  'Taxation',
  'Finance software',
  'Compliance',
  'Financial modeling',
  'Cash flow analysis',
  'Asset management',
  'Financial planning',
  'Expense tracking',
  'Regulatory knowledge',
  'Market trends',
  'Cost control',
  'Mergers and acquisitions'],
 'Admin & Office': ['Varied skills depending on the specific role'],
 'Building & Architecture': ['Varied skills depending on the specific role'],
 'Community & Social Services': ['Technical troubleshooting',
  'System administration',
  'Help desk support',
  'IT security protocols',
  'Hardware repair',
  'Network monitoring',
  'Software installation',
  'Technical documentation',
  'Operating systems',
  'Technical training',
  'Software development support',
  'IT infrastructure',
  'Troubleshooting software',
  'Network configurations',
  'IT security awareness

In [104]:
#apy_data = df_sample["Job_Skills"].to_list()
apy_data = df_sorted["Job_Skills"].to_list()

In [105]:
import mlxtend
#import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
te=TransactionEncoder()
te_ary=te.fit(apy_data).transform(apy_data)    #Apply one-hot-encoding on our dataset
apy_df=pd.DataFrame(te_ary, columns=te.columns_)

In [106]:
apy_df

,3D modeling,Account management,Accounting,Adaptability,Adaptation to change,Administrative software,Administrative support,Adobe Creative Suite,Advertising,Advisory services,...,Vulnerability assessment,Warehouse management,Warehouse organization,Wealth management,Web design,Wellness counseling,Work ethic,Workforce planning,Workplace safety,Writing proficiency
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2925,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2926,False,False,False,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2927,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [108]:
from mlxtend.frequent_patterns import apriori
apriori(apy_df, min_support=0.15, use_colnames= True).sort_values("support")

,support,itemsets
4,0.150222,(Negotiation)
32,0.163878,"(Sales strategy, Product knowledge, Competitiv..."
35,0.163878,"(Market research, Sales forecasting, Lead gene..."
36,0.163878,"(Sales strategy, Market research, Lead generat..."
37,0.163878,"(Product knowledge, Lead generation, Sales for..."
...,...,...
1,0.179583,(Competitive analysis)
5,0.193923,(Problem-solving)
6,0.195288,(Product knowledge)
3,0.200410,(Market research)


In [117]:
support_df = apriori(apy_df, min_support=0.001, use_colnames= True, max_len=3, verbose = 2)
#Combinations higher than 3 lead to memory error.

Processing 3233703 combinations | Sampling itemset size 3


In [118]:
support_df

,support,itemsets
0,0.032776,(3D modeling)
1,0.026630,(Account management)
2,0.065893,(Accounting)
3,0.079208,(Adaptability)
4,0.024582,(Adaptation to change)
...,...,...
43134,0.012291,"(Visual storytelling, Typography, Web design)"
43135,0.012291,"(Visual storytelling, User interface design, V..."
43136,0.012291,"(User interface design, Visual communication, ..."
43137,0.012291,"(Visual storytelling, User interface design, W..."


In [119]:
from mlxtend.frequent_patterns import association_rules 

lift_df = association_rules(support_df,metric="lift",min_threshold=1)#.sort_values("zhangs_metric", ascending= False) # associate items


In [120]:
lift_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(AutoCAD),(3D modeling),0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
1,(3D modeling),(AutoCAD),0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
2,(3D modeling),(Blueprint reading),0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
3,(Blueprint reading),(3D modeling),0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
4,(3D modeling),(Building codes),0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
231969,"(Visual storytelling, Web design)",(Visual communication),0.012291,0.012291,0.012291,1.0,81.361111,0.012140,inf,1.0
231970,"(Visual communication, Web design)",(Visual storytelling),0.012291,0.012291,0.012291,1.0,81.361111,0.012140,inf,1.0
231971,(Visual storytelling),"(Visual communication, Web design)",0.012291,0.012291,0.012291,1.0,81.361111,0.012140,inf,1.0
231972,(Visual communication),"(Visual storytelling, Web design)",0.012291,0.012291,0.012291,1.0,81.361111,0.012140,inf,1.0


In [121]:
print(f"There are {len(lift_df)} association rules to use")

There are 231974 association rules to use


In [127]:
import pandas as pd

k = pd.read_csv("Association_rules.csv")
k.to_parquet("Association_rules.parquet")

In [150]:
"'abc'".replace("'", "")

'abc'

In [151]:
kdf = pd.read_parquet("Association_rules.parquet")

kdf["antecedents"] = kdf["antecedents"].str.strip("frozenset").str.strip("({})").str.replace("'", "")

kdf["consequents"] = kdf["consequents"].str.strip("frozenset").str.strip("({})").str.replace("'", "")

kdf

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,AutoCAD,3D modeling,0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
1,3D modeling,AutoCAD,0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
2,3D modeling,Blueprint reading,0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
3,Blueprint reading,3D modeling,0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
4,3D modeling,Building codes,0.032776,0.032776,0.032776,1.0,30.510417,0.031701,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
231969,"Visual storytelling, Web design",Visual communication,0.012291,0.012291,0.012291,1.0,81.361111,0.012140,inf,1.0
231970,"Visual communication, Web design",Visual storytelling,0.012291,0.012291,0.012291,1.0,81.361111,0.012140,inf,1.0
231971,Visual storytelling,"Visual communication, Web design",0.012291,0.012291,0.012291,1.0,81.361111,0.012140,inf,1.0
231972,Visual communication,"Visual storytelling, Web design",0.012291,0.012291,0.012291,1.0,81.361111,0.012140,inf,1.0


In [152]:
import numpy as np
np.where(kdf["antecedents"].str.startswith('Visual storytelling'))

(array([   368,    700,    844,   1013,   1686,   2482,   3168,   4380,
          5881,   7456,   7962,   7978,   8756,  10140,  10198,  11742,
         11748,  11756,  11760,  21742,  21743,  21745,  21844,  21845,
         21847,  21942,  21945,  22030,  22031,  22033,  22114,  22115,
         22117,  22192,  22193,  22195,  22264,  22265,  22267,  22332,
         22335,  22390,  22391,  22393,  22444,  22445,  22447,  22492,
         22493,  22495,  22534,  22535,  22537,  22570,  22571,  22573,
         22600,  22601,  22603,  22624,  22625,  22627,  22642,  22643,
         22645,  22654,  22655,  22657,  22666,  22667,  22669,  30538,
         30539,  30541,  30636,  30639,  30724,  30725,  30727,  30808,
         30809,  30811,  30888,  30891,  30958,  30959,  30961,  31026,
         31029,  31084,  31085,  31087,  31138,  31139,  31141,  31186,
         31187,  31189,  31228,  31229,  31231,  31264,  31265,  31267,
         31296,  31299,  31318,  31319,  31321,  31338,  31341, 

In [153]:
kdf.to_parquet("Finished_Association_rules.parquet")

In [113]:
ante = lift_df["antecedents"].to_list()

In [114]:
ant_list = []

for ant in ante:
    ant_list.extend(list(ant))
    
len(set(ant_list))

532

In [81]:
l = []
for key in skills_ready.keys():
    l.extend(skills_ready[key])

set_l = set(l)

len(set_l)


590

In [68]:
df["Industry"].value_counts()

Sales                                  294
Accounting, Auditing & Finance         287
Marketing & Communications             272
Research, Teaching & Training          228
Admin & Office                         214
Engineering & Technology               201
Management & Business Development      154
Human Resources                        146
Software & Data                        139
Driver & Transport Services            112
Trades & Services                      112
Medical & Pharmaceutical               100
Food Services & Catering                87
Legal Services                          82
Health & Safety                         78
Supply Chain & Procurement              76
Creative & Design                       71
Customer Service & Support              68
Hospitality & Leisure                   60
Estate Agents & Property Management     44
Building & Architecture                 36
Product & Project Management            28
Farming & Agriculture                   22
Community &

In [79]:
industry_skills = {}

for industry in df["Industry"].unique():
    sales_mask = df["Industry"] == industry
    industry_skills[industry] = []
    industry_skills[industry].extend(df.loc[sales_mask, :]["Job_Skills"].to_list())

#df.loc[sales_mask, :]["Job_Skills"].to_list()

In [81]:
industry_skills.keys()

dict_keys(['Accounting, Auditing & Finance', 'Marketing & Communications', 'Health & Safety', 'Human Resources', 'Medical & Pharmaceutical', 'Food Services & Catering', 'Driver & Transport Services', 'Management & Business Development', 'Admin & Office', 'Quality Control & Assurance', 'Sales', 'Engineering & Technology', 'Software & Data', 'Creative & Design', 'Building & Architecture', 'Supply Chain & Procurement', 'Hospitality & Leisure', 'Research, Teaching & Training', 'Trades & Services', 'Estate Agents & Property Management', 'Customer Service & Support', 'Product & Project Management', 'Farming & Agriculture', 'Legal Services', 'Community & Social Services', 'Consulting & Strategy'])

In [84]:
len(industry_skills['Marketing & Communications'])

272

In [2]:
barber = industry_skills['Marketing & Communications']

NameError: name 'industry_skills' is not defined

In [108]:
#barber = industry_skills['Marketing & Communications']
for item in barber:
    if not isinstance(item, list):
        barber.remove(item)
        
import mlxtend
#import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
te=TransactionEncoder()
te_ary=te.fit(barber).transform(barber)    #Apply one-hot-encoding on our dataset
df=pd.DataFrame(te_ary, columns=te.columns_)

In [98]:
for item in barber:
    if not isinstance(item, list):
        barber.remove(item)
        

In [107]:
for item in barber:
    if not isinstance(item, list):
        print(item)
        barber.remove(item)


In [128]:
from mlxtend.frequent_patterns import apriori
apriori(df, min_support=0.3, use_colnames= True).sort_values("support")

,support,itemsets
0,0.423423,(Advertising)
699041,0.423423,"(Online presence, Advertising, Social media ad..."
699042,0.423423,"(Online presence, Advertising, Content strateg..."
699043,0.423423,"(Online presence, Advertising, Content strateg..."
699044,0.423423,"(Advertising, Content strategy, Influencer mar..."
...,...,...
579,0.513514,"(Content creation, Audience engagement, Conten..."
3451,0.513514,"(Content creation, Audience engagement, Conten..."
120,0.513514,"(Content creation, Copywriting)"
132,0.513514,"(Content strategy, Copywriting)"


In [135]:
support_df = apriori(df, min_support=0.4, use_colnames= True).sort_values("support")

In [ ]:
from mlxtend.frequent_patterns import association_rules 

lift_df = association_rules(support_df,metric="lift",min_threshold=1.1).sort_values("zhangs_metric", ascending= False) # associate items


In [139]:
support_df

,support,itemsets
0,0.423423,(Advertising)
699041,0.423423,"(Online presence, Advertising, Social media ad..."
699042,0.423423,"(Online presence, Advertising, Content strateg..."
699043,0.423423,"(Online presence, Advertising, Content strateg..."
699044,0.423423,"(Advertising, Content strategy, Influencer mar..."
...,...,...
579,0.513514,"(Content creation, Audience engagement, Conten..."
3451,0.513514,"(Content creation, Audience engagement, Conten..."
120,0.513514,"(Content creation, Copywriting)"
132,0.513514,"(Content strategy, Copywriting)"
